In [1]:
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import FirefoxOptions

def get_html(url, driver=None):
    _driver = driver
    if driver is None:
        _driver = webdriver.Firefox()
    _driver.get(url)
    html = _driver.find_element_by_tag_name("html").get_attribute("innerHTML")
    if driver is None:
        _driver.close()
    return html

In [3]:
opts = FirefoxOptions()
opts.headless = True
driver = webdriver.Firefox(options=opts)
print("entering")
raw_html = get_html("https://www.worldometers.info/coronavirus/country/us/", driver=driver)
driver.close()

entering


In [4]:
dfs = pd.read_html(raw_html)

In [5]:
df = dfs[0]
df.columns = ["state", "cases", "new_cases", "deaths", "new_deaths", "active_cases", "case_density", "death_density", "tests", "test_density", "source"]

idx = df.state.apply(lambda x: any(t in x.lower() for t in ["total", "wuhan", "princess", "islands", "rico", "guam"]))
df = df[~idx].copy()
df["positive_test_rate"] = 100 * df["cases"] / df["tests"]

df.head()

,state,cases,new_cases,deaths,new_deaths,active_cases,case_density,death_density,tests,test_density,source,positive_test_rate
1,New York,123018,NaN,4159.0,NaN,106672,6271.0,212.0,302280,15408.0,[1] [2] [3] [4] [5] [6] [7] [8] [9] [10],40.696705
2,New Jersey,37505,NaN,917.0,NaN,36496,4223.0,103.0,82166,9251.0,[1] [2],45.645401
3,Michigan,15718,NaN,617.0,NaN,15032,1579.0,62.0,45748,4594.0,[1] [2] [3],34.357786
4,California,15201,+164,350.0,3.0,13951,388.0,9.0,116533,2977.0,[1] [2],13.044374
5,Florida,13324,+974,236.0,15.0,12988,647.0,11.0,116898,5675.0,[1] [2],11.397971


In [6]:
df.dtypes

state                  object
cases                   int64
new_cases              object
deaths                float64
new_deaths            float64
active_cases            int64
case_density          float64
death_density         float64
tests                   int64
test_density          float64
source                 object
positive_test_rate    float64
dtype: object

Want a time series:

* x: n_tests
* y: positive_test_rate
* (curve parameterized by time)
* select: state

In [7]:
import altair as alt

alt.Chart(df).mark_text().encode(
    x=alt.X("tests", scale=alt.Scale(type="log")),
    y="positive_test_rate",
    text="state",
#     color="positive_test_rate",
)

alt.Chart(...)

In [8]:
opts = FirefoxOptions()
opts.headless = True
driver = webdriver.Firefox(options=opts)
print("entering")
raw_html = get_html("https://www.worldometers.info/coronavirus/covid-19-testing/", driver=driver)
driver.close()

entering


In [9]:
dfs = pd.read_html(raw_html)

In [10]:
dfs

[              0                1                         2           3  \
 0       Country  Tests Performed  Tests per Million People  Population   
 1       Bahrain             8354                      4910     1701575   
 2   South Korea           210144                      4099    51269185   
 3     Hong Kong            16000                      2134     7496981   
 4         Italy            60761                      1005    60461826   
 5   Switzerland             5000                       578     8654622   
 6       Austria             5026                       558     9006398   
 7            UK            26261                       387    67886011   
 8       Belgium             3984                       344    11589623   
 9     Australia             8278                       325    25499884   
 10       Israel             2386                       276     8655535   
 11       France            11895                       182    65273511   
 12      Finland         